# Internal control experiments

This notebooks runs some experiments with internal bandit NRM control. 

For the application of the NRM model to resource management to one
computational job, the global resource optimization problem is the following:

$$
\begin{array}{l}
    \min \quad e_{\text{total}} \\
	\text{s.t.} \quad  t > \tau t_{\text{ref}}
\end{array}
$$

Where $e_{\text{total}}$ denotes the total energy spent by the system during
the lifetime of the job, whose duration is denoted by $t^T$. We denote by
$t_{\text{ref}}$ a reference measurement of the runtime of the job on an
unmanaged system. $\tau <1$ is a parameter controlling the amount of runtime
degradation allowed for the job.

The value of this global objective can be easily measured a-posteriori for a
computational job using power instrumentation techniques. Assuming both
workload and platform behavior to be deterministic, this objective is measured
using two runs of the system: A first run without resource management to
acquire $t_{\text{ref}}$, and one run with NRM enabled. In order for NRM's
round-based control strategy to address this problem, we need an online loss
value however. This loss is obtained using the following loose assumptions:

- The passive power consumption of the node is fixed and known. [1]

- The total power consumption in a given time period can be estimated as
  the sum of the static node consumption over that period and the RAPL power
  measurement over that period. [2]

- The impact of a choice of power-cap on the job's runtime can be
  interpolated linearly from its impact on CPU counters. [3]


Denoting as in the previous section the round counter by $0<r<T$, the known
passive static power consumption by $p_{\text{static}}$, the starting time of
the job by $t^0$ and the end time of round $r$ by $t^r$, we can write the total
energy expenditure of the job based on RAPL power measurements $p^r$ using
assumptions 1 and 2 as:

$$
	e_{\text{total}} = \sum_{r=1}^{r=T} (p^r + p_{\text{static}}) (t^{r-1} - t^{r})
$$

Using assumption 3 means that we can reasonably estimate the
change in job runtime incurred by the choice of power-cap in round $r$ by
evaluating $\frac{s^r_{\text{ref}}}{s^r}$. We use this as part of our proxy
cost in two ways. First, this quantity is used to evaluate breaching of the
constraint on $t$, and second, it is used to adjust for an expected increase in
the number of rounds due to the impact on job runtime. This gives rise to the
following value for the loss at round $r$:

$$
	\ell^r = \mathbb{\huge 1}_{\left( \frac{s^r}{s^r_{\text{ref}}}>\tau \right)}
   \left( \frac{s^r_{\text{ref}}}{s^r} \left( p^r + p_{\text{static}} \right) \right)
$$


In [1]:
cd ..

/home/fre/workspace/hnrm


In [ ]:
%%capture
%%bash
./shake.sh build # for the daemon 
./shake.sh client # for the upstream client
./shake.sh pyclient # for the shared client libraryt

In [ ]:
%%bash
dhall-to-json <<< '
    let 
        Cfg = ./resources/types/Cfg.dhall
    in 
        (./resources/defaults/Cfg.dhall // {controlCfg = Some {
         minimumControlInterval = {fromuS = 1000000.0},
         learnCfg = < Lagrange :{lagrangeConstraint: Double}|
                      Knapsack :{knapsackConstraint: Double} >.Lagrange {
                        lagrangeConstraint = 1.0 
                      },
          speedThreshold = 0.9,
        referenceMeasurementRoundInterval = +10
        }}):Cfg'

In [ ]:
%load_ext nb_black
import json

daemonCfg = {
    "controlCfg": {
        "referenceMeasurementRoundInterval": 10,
        "learnCfg": {"lagrangeConstraint": 1},
        "speedThreshold": 0.9,
        "minimumControlInterval": {"fromuS": 1000000},
    },
    "verbose": "Info",
}


print(daemonCfg)

workloads = {
    "dummy": [
        {
            "cmd": "stream_c",
            "args": [],  # running the sleep command for 10 seconds.
            "sliceID": "toto",
            "manifest": {
                "app": {
                    "slice": {"cpus": 1, "mems": 1},
                    "perfwrapper": {
                        "perfLimit": {"fromOps": 100000},
                        "perfFreq": {"fromHz": 1},
                    },
                },
                "name": "perfwrap",
            },
        }
    ],
    # "other":your code
}

In [ ]:
import nrm.tooling as nrm

host = nrm.Local()

In [ ]:
host.start_daemon(daemonCfg)
assert host.check_daemon()

cpd = host.get_cpd()
print("Pretty-printing the control problem description:\n %s" % cpd)
cpdd = dict(cpd)
print("Python dictionary version: %s" % cpdd)

In [ ]:
s = host.get_state()
print(s)
print(dict(s))

The next cell just stops the daemon cleanly.

In [ ]:
host.stop_daemon()
assert host.check_daemon() == False

### Dummy experiment: printing measurements

We now are ready to run an external resource management strategy. The next cell uses the low-level "upstream" message passing interface to run a dummy strategy that takes a constant action. Message schemas for this API can be browsed in the [./resources/](./resources/) folder.

In [ ]:
import time

for name, workload in workloads.items():
    host.start_daemon(daemonCfg)
    print("Starting the workload")
    host.run_workload(workload)
    history = {}
    print(host.get_state())
    getCPD = True
    while host.check_daemon() and not host.workload_finished():
        measurement_message = host.workload_recv()
        msg = json.loads(measurement_message)
        #print(msg)
        if "pubMeasurements" in msg:
            if getCPD:
                getCPD = False
                time.sleep(1)
                cpd = dict(host.get_cpd())
                print("Sensor identifier list:")
                for sensorID in [sensor[0] for sensor in cpd["sensors"]]:
                    print("- %s" % sensorID)
                print("Actuator identifier list:")
                for sensorID in [sensor[0] for sensor in cpd["actuators"]]:
                    print("- %s" % sensorID)
            content = msg["pubMeasurements"][1][0]
            t = content["time"]
            sensorID = content["sensorID"]
            x = content["sensorValue"]
            print(
                "Received measurement originating at time %s for sensor %s of value %s"
                % (content["time"], content["sensorID"], content["sensorValue"])
            )
            if sensorID in history:
                history[sensorID].append((t, x))
            else:
                history[sensorID] = [(t, x)]
    host.check_daemon()
host.stop_daemon()

Let's plot those sensor measurements. Not very interesting since our sleep job did not produce any operations, but here goes:

In [ ]:
import pandas as pd

print(history)

for sensorID, measurements in history.items():
    dataframe = pd.DataFrame(t
        data=[(pd.Timestamp(t, unit="us"), m) for t, m in measurements]
    )
    dataframe.columns = ["time", "value"]
    dataframe = dataframe.set_index("time")
    ax = dataframe.plot(
        marker="o", linestyle="-", title='values for sensor "%s[..]"' % sensorID[:20]
    )

In [ ]:
ax